In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/media/paulati/Nuevo vol/paula/dh/2021/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


Los temas que vimos en esta parte son limpieza de datos, expresiones regulares, funciones lambda, apply .

La idea de esta práctica es ejercitar los puntos que vamos a necesitar en la clase presencial.

Usaremos el dataset de las obras del Met (The Metropolitan Museum of Art)

https://github.com/metmuseum/openaccess/



## Ejercicio 1
Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos de qué tipos de datos son las columnas.


In [2]:
import pandas as pd

# local
data_location = "../Data/MetObjects_sample.csv"

data = pd.read_csv(data_location)

data.dtypes

Unnamed: 0                   int64
Object Number               object
Is Highlight                  bool
Is Public Domain              bool
Is Timeline Work              bool
Object ID                    int64
Department                  object
AccessionYear               object
Object Name                 object
Title                       object
Culture                     object
Period                      object
Dynasty                     object
Reign                       object
Portfolio                   object
Artist Role                 object
Artist Prefix               object
Artist Display Name         object
Artist Display Bio          object
Artist Suffix               object
Artist Alpha Sort           object
Artist Nationality          object
Artist Begin Date           object
Artist End Date             object
Artist Gender               object
Artist ULAN URL             object
Artist Wikidata URL         object
Object Date                 object
Object Begin Date   

## Ejercicio 2
¿Qué campos tienen valores nulos? ¿Qué porcentaje de nulos tienen cada uno de ellos?

In [3]:
cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis=0)
print(cant_nulos_por_campo)

Unnamed: 0                    0
Object Number                 0
Is Highlight                  0
Is Public Domain              0
Is Timeline Work              0
Object ID                     0
Department                    0
AccessionYear              1000
Object Name                  88
Title                       829
Culture                    2425
Period                     3424
Dynasty                    4559
Reign                      4672
Portfolio                  4599
Artist Role                2359
Artist Prefix              4007
Artist Display Name        2345
Artist Display Bio         2631
Artist Suffix              4664
Artist Alpha Sort          2345
Artist Nationality         3169
Artist Begin Date          2646
Artist End Date            2655
Artist Gender              3751
Artist ULAN URL            2964
Artist Wikidata URL        3430
Object Date                1570
Object Begin Date             0
Object End Date               0
Medium                      132
Dimensio

In [4]:
cant_registros = data.shape[0]
print(cant_registros)

porc_nulos_por_campo = 100 * cant_nulos_por_campo / cant_registros
print(porc_nulos_por_campo)

4743
Unnamed: 0                   0.000000
Object Number                0.000000
Is Highlight                 0.000000
Is Public Domain             0.000000
Is Timeline Work             0.000000
Object ID                    0.000000
Department                   0.000000
AccessionYear               21.083702
Object Name                  1.855366
Title                       17.478389
Culture                     51.127978
Period                      72.190597
Dynasty                     96.120599
Reign                       98.503057
Portfolio                   96.963947
Artist Role                 49.736454
Artist Prefix               84.482395
Artist Display Name         49.441282
Artist Display Bio          55.471221
Artist Suffix               98.334388
Artist Alpha Sort           49.441282
Artist Nationality          66.814253
Artist Begin Date           55.787476
Artist End Date             55.977230
Artist Gender               79.084967
Artist ULAN URL             62.492094
Artist 

## Ejercicio 3

Analicemos la columna Object Date

¿Qué formato tienen los valores de este campo? ¿Qué patrones pueden identificar?

Sugerencia: recuerden el método value_counts
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html

In [5]:
object_date_series = data["Object Date"]
object_date_series

0                            20th century
1                                    1642
2                                    1911
3                                    1909
4                           June 20, 1867
                      ...                
4738                    1782, 2nd edition
4739                             ca. 1938
4740    3rd century B.C.–A.D. 4th century
4741                                 1669
4742                             ca. 1925
Name: Object Date, Length: 4743, dtype: object

Vemos que hay campos con el sufijo "th century" cuyo valor entero es un siglo, periodos de tiempo identidficados por la presencia de -, un prefijo "last quarter".

Entonces, en una misma columna tenemos años, períodos, siglos y cuartos de siglo.

## Ejercicio 4

Usando expresiones regulares, apply y funciones lambda, creemos un nuevo campo "ObjectDateClean" que sea de tipo numérico y tenga el valor del año corresponiente a cada registro extrayéndolo del campo ObjectDate.

¿Qué características tienen los valores de los registros que no verificaron ningún patrón de los definidos?

Ayuda: Identifiquemos los patrones y decidamos cómo extraer el año de los valores de los registros que verifican cada uno de esos patrones. Después, de a un patrón, usemos la estrategia que definimos antes para extraer el valor del año de los registros que lo verifican.

Vamos a empezar identificando los nulos en ese campo, que sabemos (por el resultado del ejercicio 1) son 160

In [6]:
null_mask = object_date_series.isnull()
null_mask.sum()

1570

Vamos a identificar ahora los enteros, que representan año. Para eso vamos a definir una expresión regular.

https://docs.python.org/3/library/re.html#match-objects

In [7]:
import re 
import numpy as np 

year_pattern = "(\d)?\d\d\d$"
year_pattern_regex = re.compile(year_pattern)

data_year_match = object_date_series.apply(lambda x:  x if x is np.NaN else year_pattern_regex.search(x))

mask_data_year_match_notnull = data_year_match.notnull()

data.loc[mask_data_year_match_notnull, "ObjectDateClean"] = data_year_match[mask_data_year_match_notnull].apply(lambda x: x.group(0))

#len(data_year)

Veamos cómo quedaron en los valores que estaban expresados en años (sin cambios):

In [8]:
data.loc[mask_data_year_match_notnull, ["Object Date", "ObjectDateClean"]]

,Object Date,ObjectDateClean
1,1642,1642
2,1911,1911
3,1909,1909
4,"June 20, 1867",1867
5,ca. 1878,1878
...,...,...
4733,1900–1907,1907
4734,1883,1883
4739,ca. 1938,1938
4741,1669,1669


Vamos a identificar ahora los períodos, quedándonos con el año desde. 

In [9]:
period_pattern = "(?P<year_from>\d\d\d\d)\-(\d)+"
period_pattern_regex = re.compile(period_pattern)

data_period_match = object_date_series.apply(lambda x: x if x is np.NaN else period_pattern_regex.search(x))

mask_data_period_match_notnull = data_period_match.notnull()

data.loc[mask_data_period_match_notnull, "ObjectDateClean"] = data_period_match[mask_data_period_match_notnull].apply(lambda x: x.group("year_from"))


Veamos cómo quedaron en años los valores que estaban expresados en períodos:

In [10]:
data.loc[mask_data_period_match_notnull, ["Object Date", "ObjectDateClean"]]

,Object Date,ObjectDateClean
133,1890-1910,1890
142,ca. 1850-70,1850
163,1916-1917,1916
253,1917-1918,1917
275,1916-1917,1916
367,1916-1917,1916
495,1916-1917,1916
551,1916-1917,1916
555,1916-1917,1916
846,1916-1917,1916


Vamos a identificar ahora a los siglos:

In [11]:
century_pattern = "(?P<century>\d\d)(th (C|c)entury)"
century_pattern_regex = re.compile(century_pattern)

data_century_match = object_date_series.apply(lambda x: x if x is np.NaN else century_pattern_regex.search(x))

mask_century_period_match_notnull = data_century_match.notnull()

data_century = data_century_match[mask_century_period_match_notnull].apply(lambda x: x.group("century"))

#print(data_century)
#len(data_century)

#year_from_century = (data_century.astype(int) - 1) * 100
#year_from_century

#year_to_century = (data_century.astype(int) * 100) - 1
#year_to_century

data.loc[mask_century_period_match_notnull, "ObjectDateClean"] = \
    data_century_match[mask_century_period_match_notnull].apply(lambda x: x.group("century"))

data.loc[mask_century_period_match_notnull, "ObjectDateClean"] = \
    (data.loc[mask_century_period_match_notnull, "ObjectDateClean"].astype(int) - 1) * 100


Veamos cómo quedaron en años los valores que estaban expresados en siglos:

In [12]:
data.loc[mask_century_period_match_notnull, ["Object Date", "ObjectDateClean"]]

,Object Date,ObjectDateClean
0,20th century,1900
9,11th–13th century,1200
26,19th century,1800
27,17th century,1600
29,19th century,1800
...,...,...
4708,late 19th century,1800
4710,18th century,1700
4728,first half 18th century,1700
4730,early 13th century,1200


Reemplazemos todos los valores nulos por 0, y verifiquemos cómo queda

In [13]:
mask_null = object_date_series.isnull()

data.loc[mask_null, "ObjectDateClean"] = 0

data.loc[mask_null, ["Object Date", "ObjectDateClean"]]


,Object Date,ObjectDateClean
51,NaN,0
103,NaN,0
122,NaN,0
136,NaN,0
159,NaN,0
...,...,...
2498,NaN,0
2499,NaN,0
2500,NaN,0
2501,NaN,0


Veamos qué características tienen los valores de los registros que nos quedaron en null en el campo ObjectDateClean

In [14]:
mask_null_ObjectDateClean = data["ObjectDateClean"].isnull()

print(mask_null_ObjectDateClean.sum())

data.loc[mask_null_ObjectDateClean, ["Object Date", "ObjectDateClean"]]


1081


,Object Date,ObjectDateClean
8,1815–30,NaN
10,ca. 1808–12,NaN
11,ca. 1295–1070 B.C.,NaN
12,1815–30,NaN
15,1st–2nd century A.D.,NaN
...,...,...
4735,ca. 2124–1504 B.C.,NaN
4736,1825–35,NaN
4737,4th–3rd century B.C.,NaN
4738,"1782, 2nd edition",NaN


In [15]:
no_match_data = data["Object Date"].iloc[0]
print(ascii(no_match_data[8]))
print(ascii("-"))

't'
'-'


Vemos que nos faltó considerar:
* los valores terminados en BC o B.C., y marcar esos años como negativos 
* considerar los períodos que tienen siglos, en lugar de años
* borrar las C. que aparecen de sufijo
* Considerar pallabras mals escritas como "cenutry"

Si tienen ganas de seguir practicando pueden resolver algunos de éstos.
